# Plotting tutorial

The plotting module is based on the [Bokeh](http://bokeh.pydata.org/en/latest/) package and enables visualization of various properties of elements. The plotting in the form of periodic table is done by the `periodic_plot` function that takes a [pandas](http://pandas.pydata.org/) `DataFrame`. 

To embed the plots in the [jupyter notebook](http://jupyter.org/) first the `BokehJS` needs to be started.

In [32]:
%matplotlib inline

In [2]:
! pip install bokeh

     |████████████████████████████████| 17.6MB 79kB/s  eta 0:00:01
     |████████████████████████████████| 276kB 40.9MB/s eta 0:00:01
     |████████████████████████████████| 2.0MB 37.6MB/s eta 0:00:01
     |████████████████████████████████| 71kB 4.1MB/s eta 0:00:011
  Stored in directory: /home/jovyan/.cache/pip/wheels/96/8c/18/ab51f7028839c79738fc7b21c7d660f3d59e7748eb903fbe15
  Stored in directory: /home/jovyan/.cache/pip/wheels/ad/56/bc/1522f864feb2a358ea6f1a92b4798d69ac783a28e80567a18b
Successfully built bokeh PyYAML


In [6]:
!pip install seaborn

     |████████████████████████████████| 215kB 3.3MB/s eta 0:00:01
     |████████████████████████████████| 25.2MB 56kB/s  eta 0:00:011��███             | 15.0MB 4.8MB/s eta 0:00:03
     |████████████████████████████████| 13.1MB 125kB/s eta 0:00:01    |████                            | 1.6MB 6.0MB/s eta 0:00:02ta 0:00:02
     |████████████████████████████████| 92kB 17.5MB/s eta 0:00:01


In [3]:
from bokeh.plotting import output_notebook, output_file

In [4]:
#output_notebook()

Now we can get the data from `mendeleev` as a pandas `DataFrame` through `get_table` method

In [7]:
from mendeleev import get_table
from mendeleev.plotting import periodic_plot

In [8]:
ptable = get_table('elements')

To plot the default table pass the `ptable` to the `periodic_plot` function

In [9]:
periodic_plot(ptable)

`mendeleev` stores also two color schemes for atoms that are frequently used for visualizing molecular structures. One set is stored in the `cpk_color` column and refers to [CPK](https://en.wikipedia.org/wiki/CPK_coloring) coloring, another is stored in `jmol_color` column and is used by the [Jmol program](http://jmol.sourceforge.net/jscolors/), finally there is also coloring scheme from [MOLCAS GV](http://www.molcas.org/GV/) program store in the `molcas_gv_color` attribute. They can be displayed either by hovering of the element to display a tooltip or used directly to color the element cells.

In [34]:
periodic_plot(ptable, colorby='jmol_color', title="JMol Colors", decimals=3)

In [11]:
periodic_plot(ptable, colorby='cpk_color', title='CPK Colors')

In [12]:
periodic_plot(ptable, colorby='molcas_gv_color', title='MOLCAS GV Colors')

You can also add a custom color map by assigning color to all the elments. This can be done by creating a custom column in the `DataFrame` and then using `colorby` argument to specify which column contains colors. Let's try to color the elements according to the block they belong to.

In [13]:
import seaborn as sns
from matplotlib import colors
blockcmap = {b : colors.rgb2hex(c) for b, c in zip(['s', 'p', 'd', 'f'], sns.color_palette('deep'))}
ptable['block_color'] = ptable['block'].map(blockcmap)
periodic_plot(ptable, colorby='block_color')

# Visualizing properties 

Any of the properties in `ptable` can now be visualized. As a first example lets see the `covalent_radius_pyykko` with the detault colormap, which is `RdBu_r`

In [36]:
periodic_plot(ptable, attribute='covalent_radius_pyykko', colorby='attribute', title="Covalent Radii of Pyykko")

The colormap can be changed using the `cmap` argument to any of the [standard colormaps](http://matplotlib.org/users/colormaps.html)

In [15]:
periodic_plot(ptable, attribute='covalent_radius_pyykko', colorby='attribute',
              cmap='spring', title="Covalent Radii of Pyykko")

If you have [matplotlib](http://matplotlib.org/) 1.5.0 or higher you can try our the new colormaps: `viridis`, `plasma`, `inferno` and `magma`.

In [16]:
periodic_plot(ptable, attribute='covalent_radius_pyykko', colorby='attribute',
              cmap='inferno', title="Covalent Radii of Pyykko")

To see the available properties you can access the `DataFrame`'s `columns` attribute

In [17]:
ptable.columns

Index(['annotation', 'atomic_number', 'atomic_radius', 'atomic_volume',
       'block', 'boiling_point', 'density', 'description',
       'dipole_polarizability', 'electron_affinity',
       'electronic_configuration', 'evaporation_heat', 'fusion_heat',
       'group_id', 'lattice_constant', 'lattice_structure', 'melting_point',
       'name', 'period', 'series_id', 'specific_heat', 'symbol',
       'thermal_conductivity', 'vdw_radius', 'covalent_radius_cordero',
       'covalent_radius_pyykko', 'en_pauling', 'en_allen', 'jmol_color',
       'cpk_color', 'proton_affinity', 'gas_basicity', 'heat_of_formation',
       'c6', 'covalent_radius_bragg', 'covalent_radius_slater',
       'vdw_radius_bondi', 'vdw_radius_truhlar', 'vdw_radius_rt',
       'vdw_radius_batsanov', 'vdw_radius_dreiding', 'vdw_radius_uff',
       'vdw_radius_mm3', 'abundance_crust', 'abundance_sea', 'molcas_gv_color',
       'en_ghosh', 'vdw_radius_alvarez', 'c6_gb', 'atomic_weight',
       'atomic_weight_uncertainty',

Lets try `atomic_volume`

In [18]:
periodic_plot(ptable, attribute='atomic_volume', colorby='attribute', title='Atomic Volume')

/srv/conda/envs/notebook/lib/python3.7/site-packages/matplotlib/colors.py:479: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


In [20]:
periodic_plot(ptable, attribute='en_pauling', colorby='attribute',
              title="Pauling's Electronegativity", cmap='viridis')

# Custom properties

You can also create your own properties to be displayed using [pandas](http://pandas.pydata.org/) awesome methods for manipulating data. For example let's consider the difference of electronegativity between every element and the Oxygen atom. To calculate the values we will use Allen scale this time and call our new value `ENX-ENO`.

In [21]:
ptable.loc[:, 'ENX-ENO'] =  ptable.loc[ptable['symbol'] == 'O', 'en_allen'].values - ptable.loc[:, 'en_allen']

In [22]:
periodic_plot(ptable, attribute='ENX-ENO', colorby='attribute',
              cmap='viridis', title='Allen Electronegativity wrt. Oxygen')

As a second example let's consider a difference between the `covalent_radius_slater` and `covalent_radius_pyykko` values

In [23]:
ptable['cov_rad_diff'] = ptable['covalent_radius_slater'] - ptable['covalent_radius_pyykko']

In [24]:
periodic_plot(ptable, attribute='cov_rad_diff', colorby='attribute',
              title='Covalent Radii Difference', cmap='viridis')

# Wide 32-column version

The `periodic_plot` function can also present the periodic table in the so-called wide format with the *f*-block between the *s*- and *d*-blocks resulting in 32 columns.

In [25]:
periodic_plot(ptable, height=600, width=1500, long_version=True)

In [27]:
%load_ext version_information

In [37]:
%version_information mendeleev, bokeh, pandas, matplotlib

Software versions
Python 3.7.3 64bit [GCC 7.3.0]
IPython 7.5.0
OS Linux 4.14.91+ x86_64 with debian buster sid
mendeleev 0.4.5
bokeh 1.2.0
pandas 0.24.2
matplotlib 3.1.0
Fri May 31 08:57:15 2019 UTC